In [1]:
import pandas as pd

anime = pd.read_csv('data/anime.csv', index_col='id')
pd.options.display.max_rows = 10

In [2]:
anime

,title,score,airedfrom,airedto,episodes,type,status,source,rating,members,duration,rank,popularity,favorites,description
id,,,,,,,,,,,,,,,
1,Cowboy Bebop,8.81,1998-04-03 00:00:00,1999-04-24 00:00:00,26.0,TV,Finished Airing,Original,R - 17+ (violence &amp; profanity),758864,24 min. per ep.,27,38,41714,"In the year 2071, humanity has colonized sever..."
5,Cowboy Bebop: Tengoku no Tobira,8.41,2001-09-01 00:00:00,NaN,1.0,Movie,Finished Airing,Original,R - 17+ (violence &amp; profanity),190962,1 hr. 55 min.,158,445,753,"Another day, another bounty—such is the life o..."
17,Hungry Heart: Wild Striker,7.68,2002-09-11 00:00:00,2003-09-10 00:00:00,52.0,TV,Finished Airing,Manga,PG-13 - Teens 13 or older,16739,23 min. per ep.,1156,3071,205,Kyosuke Kano has lived under the shadow of his...
18,Initial D Fourth Stage,8.23,2004-04-17 00:00:00,2006-02-18 00:00:00,24.0,TV,Finished Airing,Manga,PG-13 - Teens 13 or older,69939,27 min. per ep.,291,1254,608,Takumi Fujiwara and brothers Keisuke and Ryous...
21,One Piece,8.53,1999-10-20 00:00:00,NaN,NaN,TV,Currently Airing,Manga,PG-13 - Teens 13 or older,767265,24 min.,91,37,73849,Gol D. Roger was known as the &quot;Pirate Kin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38383,Jashin-chan Dropkick Episode 12,7.11,2018-10-01 00:00:00,NaN,1.0,ONA,Finished Airing,Web manga,PG-13 - Teens 13 or older,6799,23 min.,3211,4513,8,&quot;It&quot; was believed to be traveling in...
38395,Otona no Bouguya-san,5.79,2018-10-09 00:00:00,2018-12-25 00:00:00,12.0,ONA,Currently Airing,Web manga,PG-13 - Teens 13 or older,6100,4 min. per ep.,8134,4741,4,Based on a 4-koma web manga of the same name. ...
38411,Eiga Daisuki Pompo-san,4.96,2018-09-27 00:00:00,NaN,1.0,Special,Finished Airing,Web manga,PG-13 - Teens 13 or older,76,15 sec.,10501,13996,0,Looking for information on the anime Eiga Dais...
